# Trackers test

This notebook demonstrates tests the trackers in flax-pilot against tf.keras

**Use TensorFlow-cpu instead of TensorFlow to avoid any issues between TensorFlow & Jax.**

In [1]:
!pip install -q tensorflow-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.


In [2]:
import jax
import tensorflow as tf
import flax.linen as nn
import jax.numpy as jnp
import jax.random as jr

### Access to modules

In [24]:
import os
os.listdir("/kaggle/input/fpilot")

['trackers', 'checkpoints', 'training']

In [3]:
import sys
sys.path.append("/kaggle/input")

In [4]:
from fpilot.trackers import basic_trackers as tr

### Binary Classification Accuracy and MultiLabel Accuracy Trackers

In [5]:
tf_binary = tf.keras.metrics.BinaryAccuracy(threshold=0.7)
fp_binary = tr.BinaryAccuracy(threshold=0.7)

#### Binary

In [6]:
for i in range(50):
    true = tf.random.uniform((32, 1), minval=0, maxval=2, dtype=tf.int32).numpy()
    pred = tf.random.uniform((32, 1), minval=0, maxval=2, dtype=tf.int32).numpy()
    fp_binary.update(true, pred)
    tf_binary.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_binary.compute())
        print("TF result: ",tf_binary.result().numpy())
        tf_binary.reset_state()
        fp_binary.reset()

Result
FP result:  0.49999995000000497
TF result:  0.5
Result
FP result:  0.50937499490625
TF result:  0.509375
Result
FP result:  0.5312499946875001
TF result:  0.53125
Result
FP result:  0.5062499949375001
TF result:  0.50625
Result
FP result:  0.48124999518750006
TF result:  0.48125


#### MultiLabel

In [7]:
for i in range(50):
    true = tf.random.uniform((32, 10), minval=0, maxval=2, dtype=tf.int32).numpy()
    pred = tf.random.uniform((32, 10), minval=0, maxval=2, dtype=tf.int32).numpy()
    fp_binary.update(true, pred)
    tf_binary.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_binary.compute())
        print("TF result: ",tf_binary.result().numpy())
        tf_binary.reset_state()
        fp_binary.reset()

Result
FP result:  0.4874999951250001
TF result:  0.4875
Result
FP result:  0.5018749949812501
TF result:  0.501875
Result
FP result:  0.4996874950031251
TF result:  0.49968752
Result
FP result:  0.4965624950343751
TF result:  0.49656254
Result
FP result:  0.4946874950531251
TF result:  0.4946875


### Multiclass Accuracy Trackers

In [8]:
tf_mc = tf.keras.metrics.CategoricalAccuracy()
fp_mc = tr.Accuracy()

In [9]:
for i in range(50):
    true = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    pred = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    fp_mc.update(true, pred)
    tf_mc.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_mc.compute())
        print("TF result: ",tf_mc.result().numpy())
        tf_mc.reset_state()
        fp_mc.reset()

Result
FP result:  0.06249999375000062
TF result:  0.0625
Result
FP result:  0.09062499909375002
TF result:  0.090625
Result
FP result:  0.11249999887500002
TF result:  0.1125
Result
FP result:  0.09999999900000002
TF result:  0.1
Result
FP result:  0.07812499921875002
TF result:  0.078125


### Common Classification trackers

#### f1 macro

In [10]:
tf_f1 = tf.keras.metrics.F1Score(threshold=0.5, average='macro')
fp_f1 = tr.F1Score(num_classes=10, threshold=0.5, average='macro')

for i in range(50):
    true = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    pred = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    
    fp_f1.update(true, pred)
    tf_f1.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_f1.compute())
        print("TF result: ",tf_f1.result().numpy())
        tf_f1.reset_state()
        fp_f1.reset()

Result
FP result:  0.05333332257777953
TF result:  0.053333323
Result
FP result:  0.09924086854797884
TF result:  0.09924088
Result
FP result:  0.1122727504348547
TF result:  0.112272754
Result
FP result:  0.09665788576924698
TF result:  0.09665789
Result
FP result:  0.10827541316768115
TF result:  0.10827541


In [11]:
tf_f1.true_positives.value, tf_f1.false_positives.value, tf_f1.false_negatives.value

(<tf.Variable 'f1_score/true_positives:0' shape=(10,) dtype=float32, numpy=array([4., 4., 1., 3., 3., 3., 0., 3., 3., 3.], dtype=float32)>,
 <tf.Variable 'f1_score/false_positives:0' shape=(10,) dtype=float32, numpy=array([30., 24., 30., 25., 33., 25., 30., 13., 31., 20.], dtype=float32)>,
 <tf.Variable 'f1_score/false_negatives:0' shape=(10,) dtype=float32, numpy=array([22., 25., 16., 21., 28., 32., 34., 30., 22., 31.], dtype=float32)>)

In [12]:
fp_f1.tp , fp_f1.fp, fp_f1.fn

(array([4., 4., 1., 3., 3., 3., 0., 3., 3., 3.]),
 array([30., 24., 30., 25., 33., 25., 30., 13., 31., 20.]),
 array([22., 25., 16., 21., 28., 32., 34., 30., 22., 31.]))

#### f1 micro

In [13]:
tf_f1 = tf.keras.metrics.F1Score(threshold=0.5, average='micro')
fp_f1 = tr.F1Score(num_classes=10, threshold=0.5, average='micro')

for i in range(50):
    true = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    pred = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    fp_f1.update(true, pred)
    tf_f1.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_f1.compute())
        print("TF result: ",tf_f1.result().numpy())
        tf_f1.reset_state()
        fp_f1.reset()

Result
FP result:  0.0624999498047275
TF result:  0.062499948
Result
FP result:  0.08437494997366243
TF result:  0.08437495
Result
FP result:  0.08437494997366243
TF result:  0.08437495
Result
FP result:  0.12187494996193457
TF result:  0.12187495
Result
FP result:  0.099999949968775
TF result:  0.09999996


In [14]:
tf_f1.true_positives.value, tf_f1.false_positives.value, tf_f1.false_negatives.value

(<tf.Variable 'f1_score/true_positives:0' shape=() dtype=float32, numpy=24.0>,
 <tf.Variable 'f1_score/false_positives:0' shape=() dtype=float32, numpy=264.0>,
 <tf.Variable 'f1_score/false_negatives:0' shape=() dtype=float32, numpy=264.0>)

In [15]:
fp_f1.tp , fp_f1.fp, fp_f1.fn

(array(24.), array(264.), array(264.))

#### precision 

In [16]:
tf_p = tf.keras.metrics.Precision()
fp_p = tr.Precision(num_classes=10, threshold=0.5, average='micro')

for i in range(50):
    true = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    pred = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    fp_p.update(true, pred)
    tf_p.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_p.compute())
        print("TF result: ",tf_p.result().numpy())
        tf_p.reset_state()
        fp_p.reset()

Result
FP result:  0.15624999951171875
TF result:  0.15625
Result
FP result:  0.09062499997167968
TF result:  0.090625
Result
FP result:  0.0843749999736328
TF result:  0.084375
Result
FP result:  0.14062499995605468
TF result:  0.140625
Result
FP result:  0.0843749999736328
TF result:  0.084375


#### recall

In [17]:
tf_r = tf.keras.metrics.Recall()
fp_r = tr.Recall(num_classes=10, threshold=0.5, average='micro')

for i in range(50):
    true = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    pred = tf.one_hot(tf.random.uniform((32, ), minval=0, maxval=10, dtype=tf.int32), 10).numpy()
    fp_r.update(true, pred)
    tf_r.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_r.compute())
        print("TF result: ",tf_r.result().numpy())
        tf_r.reset_state()
        fp_r.reset()

Result
FP result:  0.124999999609375
TF result:  0.125
Result
FP result:  0.10624999996679686
TF result:  0.10625
Result
FP result:  0.10312499996777343
TF result:  0.103125
Result
FP result:  0.09062499997167968
TF result:  0.090625
Result
FP result:  0.09999999996874999
TF result:  0.1


### Mean

In [18]:
tf_mean = tf.keras.metrics.Mean()
fp_mean = tr.Mean()

for i in range(50):
    val = tf.random.uniform((32, 128), minval=1, maxval=2).numpy().mean()
    fp_mean.update(val)
    tf_mean.update_state(val)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_mean.compute())
        print("TF result: ",tf_mean.result().numpy())
        tf_mean.reset_state()
        fp_mean.reset()

Result
FP result:  1.499855845192638
TF result:  1.499856
Result
FP result:  1.501219686754771
TF result:  1.5012197
Result
FP result:  1.50015318085647
TF result:  1.5001531
Result
FP result:  1.5006420343061713
TF result:  1.5006421
Result
FP result:  1.4987221687171546
TF result:  1.4987223


### MSE

In [19]:
tf_mse = tf.keras.metrics.MeanSquaredError()
fp_mse = tr.MSE()

for i in range(50):
    true = tf.random.normal((32, 128)).numpy()
    pred = tf.random.normal((32, 128)).numpy()
    fp_mse.update(true, pred)
    tf_mse.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_mse.compute())
        print("TF result: ",tf_mse.result().numpy())
        tf_mse.reset_state()
        fp_mse.reset()

Result
FP result:  1.9981750635873397
TF result:  1.9981751
Result
FP result:  1.9891431490348077
TF result:  1.9891431
Result
FP result:  2.0079096712546742
TF result:  2.0079098
Result
FP result:  2.02157535915589
TF result:  2.0215755
Result
FP result:  1.992672173123663
TF result:  1.9926723


### MAE

In [20]:
tf_mae = tf.keras.metrics.MeanAbsoluteError()
fp_mae = tr.MAE()

for i in range(50):
    true = tf.random.normal((32, 128)).numpy()
    pred = tf.random.normal((32, 128)).numpy()
    fp_mae.update(true, pred)
    tf_mae.update_state(true, pred)
    if i%10==0:
        print("Result")
        print("FP result: ",fp_mae.compute())
        print("TF result: ",tf_mae.result().numpy())
        tf_mae.reset_state()
        fp_mae.reset()

Result
FP result:  1.0991493
TF result:  1.0991496
Result
FP result:  1.1314718
TF result:  1.1314718
Result
FP result:  1.1306143
TF result:  1.1306143
Result
FP result:  1.1297501
TF result:  1.12975
Result
FP result:  1.1218661
TF result:  1.121866
